# Progress Report

## Initial Exploration

    I started with scholar.google.com as the search engine where I would get all my data. It was the only free resource avalilable that had the full texts of judical opinions. I found the site looking for texts of SCOTUS(Supreme Court Of The United States) opinions. However, Google Scholar also had all the 2nd circuit opinions as well, published and unpublished. Thus, i decided to continue workig on my analysis of published v. unpublished opinions.

### Initial Success

    I was initialy able to webscrape a judical opinion and parse out just the text into a long string. I was also able to parse through search results (narrowed to one year 2012, at 2,110 cases) and parse out the name of the case, the url of the locatoon of the text and if it was unpublished or published(Published citations begin with at number, Unpublished begin with a letter). 

In [ ]:
def google_scholar_case_logger(search_url, n_pages=1):
    """Web scrapes google scholar for cases, needs just the /scholar? question link.
    returns a DataFrame, but should write the dataframe to a .csv as well."""
    
    google = 'https://scholar.google.com'
    final_features_list = []
    y = []
    for _ in range(0, n_pages):
        print 'Page', _
        sleep(10)
        r = requests.get(google+search_url)
        html = r.text
        soup = BeautifulSoup(html, 'lxml')
        
        final_feature = []
        for el in soup.findAll('h3'):
            sleep(30)
            print 'Building Case Row'
            
            sub_r = requests.get(google+el.a['href']) #link to the case page
            sub_html = sub_r.text
            sub_soup = BeautifulSoup(sub_html, 'lxml')
            raw_text = ''
            for x in sub_soup.findAll('p'):
                raw_text = raw_text + x.get_text() #puts all the text in one long string, raw_text
            
            final_feature.append([el.get_text(), el.a['href'], raw_text])
        for i in soup.findAll('div', {'class':'gs_a'}): #is the citation for the case
            try:
                int(i.get_text()[0]) #if the citation starts with a number, it is published
                y.append(0)
            except:
                y.append(1)
        
        final_features_list.append(final_feature)
        search_url = soup.find('div', {'id':'gs_n'}).a['href'] # the next button on the search page, links to the next page
    
    df = pd.DataFrame(final_features_list, columns=['case_name', 'case_url', 'case_text'])
    df['Unpublished'] = y
    
    return df

### Initial Obstacle

    Before I cound unite the two webscrapes togeather (I wanted a DataFrame that had [name of the case, case url, unpublished(binary), case text] as the columns), google temporarility blocked my ip requests for webscraping (but not my chrome requests). I am thus unable to obtain my entire sample at this time.

## Moving Forward

    The First thing that must be finished is the code to webscrape goodgle scholar to get the final DataFrame. I need to implement a beter way to scrape google so as to minimize disturbing them. I looked at the terms of service: https://www.google.com/policies/terms/ but I could not find any details on the best way.

### After obtaining the entire sample

    After having sampiled enough texts, I intend to do a basic nlp analysis, and select feaures that would distingiish published from unpublished.

### Forseable Problems

    I'm not sure that google scholar has all the unpublished opinions. I looked at the us court records (which publish on sept. 30th) and looked at search results (which is by Jan-Dec of any given year), and the numbers don't totaly match up. will need to look deeper to see if it is just the date causing the numbers not to match